The 'encrypt' function takes in a string. Then iterates through the letters of the string. For each letter, it takes its UTF-8 code.
Then converts it to the corresponding position of the letter in the Alphabet series. THen it plugs this number into the sine function, which gives a value between -1 and 1.
Multiply this by 26 and remove the fractional part. Then take the absolute value. The number will be the position of a letter in the Alphabet series.
Convert this into the corresponding UTF-8 code. Then convert it into a character and append to a list which will be returned.

There is another list which collects the removed fractional parts and is returned with the encrypted string. This was to be used in the decryption function, which isn't finished yet.

In [1]:
import math
import numpy as np

In [4]:
def encrypt(input_string):
    
    encrypted_string = []
    decimal_part = [] #serves as a key for decrypting
    
    for letter in input_string: #encryption is performed letter-wise independently
        x = ord(letter)
        
        decimal = 26*math.sin(x) - int(26*math.sin(x)) #the decimal part which otherwise gets lost is retained in this step
        decimal_part.append(decimal)
        
        if x > 64 and x < 91: #Upper case letters
            x -= 64
            encrypted_string.append(chr(64 + abs(int(26*math.sin(x)))))
            
        elif x > 96 and x < 123: #Lower case letters
            x -= 96
            encrypted_string.append(chr(96 + abs(int(26*math.sin(x)))))
            
        else:
            print('Invalid!')
            
    return encrypted_string, decimal_part

In [5]:
new, fract = encrypt('Hulk')

In [6]:
new

['Y', 'u', 'm', 'y']

In [7]:
fract

[0.599407431812943, -0.932146464320116, 0.09728114086241035, 0.804325358577354]

In [139]:
def decrypt(code, key): #doesnt work yet
    orig = []
    for y,f in zip(code, key):
        p = ord(y)
        if p > 64 and p < 91:
            orig.append(chr(64+int(np.arcsin(float((p-64)+f)/26.0))))
        elif p > 96 and p < 123:
            orig.append(chr(96+int(np.arcsin(float((p-96)+f)/26.0))))
    return orig

In [140]:
original = decrypt(new, fract)

In [141]:
original

['@', 'a', '`']

In [8]:
def alpha_to_code_0(ch):
    alpha_dict = {"A":0, "B":1, "C":2, "D":3, "E":4, "F":5, "G":6, "H":7, "I":8, "J":9, "K":10, "L":11, "M":12, "N":13, "O":14, "P":15, "Q":16, "R":17, "S":18, "T":19, "U":20, "V":21, "W":22, "X":23, "Y":24, "Z":25}
    return alpha_dict[ch]

In [9]:
def alpha_to_code_1(ch):
    alpha_dict = {"A":0, "B":1, "C":2, "D":3, "E":4, "F":5, "G":6, "H":7, "I":8, "J":9, "K":10, "L":11, "M":12, "N":13, "O":14, "P":15, "Q":16, "R":17, "S":18, "T":19, "U":20, "V":21, "W":22, "X":23, "Y":24, "Z":25}
    return alpha_dict[ch]+1

In [10]:
def code_to_alpha_0(ele):
    if ele > 25:
        ele -= 26
    alpha_dict = {0:"A", 1:"B", 2:"C", 3:"D", 4:"E", 5:"F", 6:"G", 7:"H", 8:"I", 9:"J", 10:"K", 11:"L", 12:"M", 13:"N", 14:"O", 15:"P", 16:"Q", 17:"R", 18:"S", 19:"T", 20:"U", 21:"V", 22:"W", 23:"X", 24:"Y", 25:"Z"}
    return alpha_dict[ele]

In [195]:
def encrypt2(st):
    new = []
    for y in st:
        x = ord(y)
        if x > 64 and x < 91:
            x -= 64
            new.append(chr(64 + abs(int(26*math.sin(math.radians(x))))))
        elif x > 96 and x < 123:
            x -= 96
            new.append(chr(96 + abs(int(26*math.sin(math.radians(x))))))
        else:
            print('Invalid!')
    print(new)

The 'Encrypt_matrix' function takes in a string, reads the first four characters, takes the corresponding position in the Alphabet Series and forms a 2x2 matrix. Then it takes the determinant of the matrix and divides the determinant by 26(total number of alphabets), let this value be 'key'. Then going through the inpuit string, it creates a new string such that the first character of the new string is a 'key' characters ahead the first character of old string, and 'key'+1 characters ahead for next character and so on.

The function returns a 'key_vector' list which can be used to decrypt the encryption

In [24]:
def Encrypt_matrix(input_string):
    
    Encrypt_string = []
    key_vector = []
    code_vector = []
    transform_matrix = [[], []]
    
    i = 0
    j = 0
    
    while i < 4:
        for letter in input_string:
            key_vector.append(alpha_to_code_0(letter))
            i+=1
    
    key_vector = np.array(key_vector)
    
    transform_matrix = key_vector[:4].reshape(2,2)
    
    det = abs(int(np.linalg.det(transform_matrix)/26))
    
    code_vector = np.add(key_vector, det)
    
    for element in code_vector:
        Encrypt_string.append(code_to_alpha_0(element + j))
        j += 1
    
    return Encrypt_string, key_vector

In [56]:
st,d = Encrypt_matrix('LEAD')

In [57]:
st

['M', 'G', 'D', 'H']

In [58]:
d

array([11,  4,  0,  3])

The 'Decrypt_matrix' takes the encrypted string and the key vector as input. It undoes the operations of the 'Encrypt_matrix' function and returns the decrypted, original string 

In [54]:
def Decrypt_matrix(encrypt_string, key_vector):
    
    decrypt_string = []
    det_matrix = [[], []]
    
    det_matrix = key_vector.reshape(2,2)
    det = abs(int(np.linalg.det(det_matrix)/26))
    
    j = 0
    for element in encrypt_string:
        code = alpha_to_code_0(element)
        if code-det-j < 0:
            code = 26-det-j+code
        else:
            code = code-det-j
        decrypt_string.append(code_to_alpha_0(code))
        j += 1
    return decrypt_string

In [59]:
Decrypt_matrix(st, d)

['L', 'E', 'A', 'D']